# isinstance() 函数详解

## 基础概念
**功能**: 检查 object 是否是 classinfo 的实例（或其子类的实例）

**语法**: `isinstance(object, classinfo)`

**返回值**: 布尔值 (True/False)

**参数说明**:
- `object`: 要检查的对象
- `classinfo`: 类、类型或类的元组

## 1. 基础用法示例

In [ ]:
# 基本数据类型检查
print(f"isinstance(42, int): {isinstance(42, int)}")
print(f"isinstance(3.14, float): {isinstance(3.14, float)}")
print(f"isinstance('hello', str): {isinstance('hello', str)}")
print(f"isinstance([1, 2, 3], list): {isinstance([1, 2, 3], list)}")
print(f"isinstance({{'a': 1}}, dict): {isinstance({'a': 1}, dict)}")

# 错误示例
print(f"isinstance(42, str): {isinstance(42, str)}")
print(f"isinstance('hello', int): {isinstance('hello', int)}")

## 2. 类和继承检查

In [ ]:
# 定义类层次结构
class Animal:
    def __init__(self, name: str):
        self.name = name

class Dog(Animal):
    def bark(self) -> None:
        print(f"{self.name} says woof!")

class Cat(Animal):
    def meow(self) -> None:
        print(f"{self.name} says meow!")

# 创建实例
my_dog = Dog("Buddy")
my_cat = Cat("Whiskers")

# isinstance 检查
print(f"isinstance(my_dog, Dog): {isinstance(my_dog, Dog)}")
print(f"isinstance(my_dog, Animal): {isinstance(my_dog, Animal)}")
print(f"isinstance(my_dog, Cat): {isinstance(my_dog, Cat)}")
print(f"isinstance(my_cat, Animal): {isinstance(my_cat, Animal)}")

print("\n--- 关键点：子类实例也是父类的实例 ---")
print(f"Dog 是 Animal 的子类，所以 my_dog 也是 Animal 的实例: {isinstance(my_dog, Animal)}")

## 3. 多类型检查（元组）

In [ ]:
# 检查对象是否属于多个类型中的任意一个
def process_number(value):
    """处理数字类型的值"""
    if isinstance(value, (int, float, complex)):
        print(f"{value} 是数字类型")
        return value * 2
    else:
        print(f"{value} 不是数字类型")
        return None

# 测试不同类型
test_values = [42, 3.14, 2+3j, "hello", [1, 2, 3], None]

for value in test_values:
    result = process_number(value)
    print(f"输入: {value}, 结果: {result}\n")

## 4. 与 type() 的区别

In [ ]:
# isinstance vs type() 的重要区别
class Parent:
    pass

class Child(Parent):
    pass

child_obj = Child()

print("=== isinstance vs type() 对比 ===")
print(f"isinstance(child_obj, Parent): {isinstance(child_obj, Parent)}")
print(f"isinstance(child_obj, Child): {isinstance(child_obj, Child)}")
print(f"type(child_obj) == Parent: {type(child_obj) == Parent}")
print(f"type(child_obj) == Child: {type(child_obj) == Child}")

print("\n关键点:")
print("- isinstance() 考虑继承关系，子类实例也是父类的实例")
print("- type() 只检查确切的类型，不考虑继承")
print("- 在面向对象编程中，推荐使用 isinstance()")

## 5. 生产级代码示例：类型安全的API处理器

In [ ]:
from typing import Union, Dict, List, Any
import json
import logging

# 配置日志
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class APIDataProcessor:
    """生产级API数据处理器，使用isinstance确保类型安全"""
    
    @staticmethod
    def validate_and_process_data(data: Any) -> Dict[str, Any]:
        """
        验证并处理API数据
        
        Args:
            data: 待处理的数据
            
        Returns:
            处理后的数据字典
            
        Raises:
            ValueError: 当数据类型不被支持时
        """
        result = {
            'original_data': data,
            'data_type': type(data).__name__,
            'processed_data': None,
            'validation_passed': False
        }
        
        try:
            # 字符串数据处理
            if isinstance(data, str):
                logger.info("处理字符串数据")
                # 尝试解析JSON
                try:
                    parsed_data = json.loads(data)
                    result['processed_data'] = parsed_data
                    result['validation_passed'] = True
                except json.JSONDecodeError:
                    # 普通字符串处理
                    result['processed_data'] = data.strip().upper()
                    result['validation_passed'] = True
            
            # 数字数据处理
            elif isinstance(data, (int, float)):
                logger.info("处理数字数据")
                result['processed_data'] = {
                    'value': data,
                    'squared': data ** 2,
                    'is_positive': data > 0
                }
                result['validation_passed'] = True
            
            # 列表数据处理
            elif isinstance(data, list):
                logger.info("处理列表数据")
                # 递归处理列表中的每个元素
                processed_list = []
                for item in data:
                    if isinstance(item, (str, int, float, bool)):
                        processed_list.append(item)
                    else:
                        processed_list.append(str(item))
                
                result['processed_data'] = {
                    'items': processed_list,
                    'count': len(processed_list),
                    'types': [type(item).__name__ for item in data]
                }
                result['validation_passed'] = True
            
            # 字典数据处理
            elif isinstance(data, dict):
                logger.info("处理字典数据")
                result['processed_data'] = {
                    'keys': list(data.keys()),
                    'values': list(data.values()),
                    'key_count': len(data)
                }
                result['validation_passed'] = True
            
            # 布尔值处理
            elif isinstance(data, bool):
                logger.info("处理布尔值")
                result['processed_data'] = {
                    'value': data,
                    'as_string': str(data),
                    'as_int': int(data)
                }
                result['validation_passed'] = True
            
            # 不支持的数据类型
            else:
                logger.warning(f"不支持的数据类型: {type(data)}")
                raise ValueError(f"不支持的数据类型: {type(data).__name__}")
                
        except Exception as e:
            logger.error(f"数据处理失败: {e}")
            result['error'] = str(e)
            result['validation_passed'] = False
        
        return result

# 测试生产级代码
processor = APIDataProcessor()

test_cases = [
    "Hello World",
    '{"name": "张三", "age": 25}',
    42,
    3.14159,
    [1, "hello", 3.14, True],
    {"user_id": 123, "username": "johndoe"},
    True,
    None,  # 这会触发错误
]

print("=== 生产级 isinstance 使用示例 ===")
for i, test_data in enumerate(test_cases, 1):
    print(f"\n--- 测试用例 {i} ---")
    print(f"输入数据: {test_data}")
    print(f"数据类型: {type(test_data).__name__}")
    
    result = processor.validate_and_process_data(test_data)
    print(f"验证通过: {result['validation_passed']}")
    
    if result['validation_passed']:
        print(f"处理结果: {result['processed_data']}")
    else:
        print(f"错误信息: {result.get('error', '未知错误')}")

## 6. 最佳实践和注意事项

In [ ]:
# 最佳实践示例

# ✅ 好的做法：使用 isinstance 进行类型检查
def safe_divide(a, b):
    """安全的除法操作"""
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError("参数必须是数字类型")
    
    if b == 0:
        raise ValueError("除数不能为零")
    
    return a / b

# ❌ 避免的做法：使用 type() 进行严格类型检查
def bad_divide(a, b):
    """不推荐的做法"""
    if type(a) not in [int, float] or type(b) not in [int, float]:
        raise TypeError("参数必须是数字类型")
    return a / b

# 测试继承情况
class MyNumber(int):
    pass

my_num = MyNumber(10)

print("=== 最佳实践对比 ===")
print(f"自定义数字类型: {my_num} (类型: {type(my_num)})")

try:
    result1 = safe_divide(my_num, 2)
    print(f"使用 isinstance() 的函数: 成功，结果 = {result1}")
except Exception as e:
    print(f"使用 isinstance() 的函数: 失败，{e}")

try:
    result2 = bad_divide(my_num, 2)
    print(f"使用 type() 的函数: 成功，结果 = {result2}")
except Exception as e:
    print(f"使用 type() 的函数: 失败，{e}")

print("\n结论: isinstance() 更灵活，支持继承关系")

## 7. 性能考虑和高级用法

In [ ]:
import timeit
from typing import Union

# 性能测试
def performance_test():
    """比较不同类型检查方法的性能"""
    test_data = [1, 2, 3, 4, 5] * 1000
    
    # isinstance 方法
    def check_with_isinstance():
        return [x for x in test_data if isinstance(x, int)]
    
    # type 方法
    def check_with_type():
        return [x for x in test_data if type(x) == int]
    
    # hasattr 方法（检查是否有某个方法）
    def check_with_hasattr():
        return [x for x in test_data if hasattr(x, '__add__')]
    
    # 测试性能
    isinstance_time = timeit.timeit(check_with_isinstance, number=1000)
    type_time = timeit.timeit(check_with_type, number=1000)
    hasattr_time = timeit.timeit(check_with_hasattr, number=1000)
    
    print("=== 性能测试结果 (1000次运行) ===")
    print(f"isinstance(): {isinstance_time:.4f} 秒")
    print(f"type():       {type_time:.4f} 秒")
    print(f"hasattr():    {hasattr_time:.4f} 秒")
    
    print("\n结论: isinstance() 通常性能很好，且语义更清晰")

performance_test()

# 高级用法：鸭子类型检查
print("\n=== 鸭子类型 vs isinstance ===")

class Duck:
    def quack(self):
        return "Quack!"

class Dog:
    def quack(self):  # 狗也会"quack"
        return "Woof! (pretending to quack)"

def make_it_quack_isinstance(animal):
    """使用 isinstance 的严格类型检查"""
    if isinstance(animal, Duck):
        return animal.quack()
    else:
        return "This is not a duck!"

def make_it_quack_duck_typing(animal):
    """使用鸭子类型的灵活检查"""
    if hasattr(animal, 'quack') and callable(getattr(animal, 'quack')):
        return animal.quack()
    else:
        return "Can't quack!"

duck = Duck()
dog = Dog()

print(f"鸭子用isinstance: {make_it_quack_isinstance(duck)}")
print(f"狗用isinstance: {make_it_quack_isinstance(dog)}")
print(f"鸭子用鸭子类型: {make_it_quack_duck_typing(duck)}")
print(f"狗用鸭子类型: {make_it_quack_duck_typing(dog)}")

print("\n总结: isinstance 更严格，鸭子类型更灵活")

## 总结

### isinstance() 的核心特点:

1. **继承感知**: 子类实例也被认为是父类的实例
2. **多类型支持**: 可以同时检查多个类型 `isinstance(obj, (int, float, str))`
3. **类型安全**: 比字符串比较或其他方法更安全
4. **性能良好**: 内置函数，经过优化

### 最佳实践:

✅ **推荐使用**:
- 参数验证
- 类型检查
- 多态处理

❌ **避免使用**:
- 过度的类型检查（违背鸭子类型原则）
- 可以用多态解决的场景

### 记忆要点:
- `isinstance()` > `type()` （考虑继承）
- 元组支持多类型检查
- 生产环境中用于参数验证
- 配合异常处理确保代码健壮性